# Web Scraping

### ~Things to do~
1. ~Scrape the website~
2. ~Get the list of files of Production data~
3. ~Get the list of urls from that section~
4. ~Go to each url and get the list of .gz files~
5. ~Download each .gz file - folder name is different for every one of them \<td\> name~

https://www.rrc.state.tx.us/about-us/resource-center/research/data-sets-available-for-download/

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
from ftplib import FTP
import os
import urllib.request as req
import shutil
from contextlib import closing

In [2]:
# This is for Jupyter notebook only
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

In [3]:
# Download the page in memory first
url = 'https://www.rrc.state.tx.us/about-us/resource-center/research/data-sets-available-for-download/'
page = requests.get(url)

In [4]:
table_data = pd.DataFrame()
if page.status_code == 200:
    soup = bs(page.content, 'lxml')
    # Upon inspection of the page - data to be extracted is in id="production-data-table" 
    tables = soup.select('table') # Reading all the table tags
    for table in tables:
        table_head = table.find('thead') # Get the head tag 
        header_rows = table_head.find_all('tr') # Get the tr 
        for header_row in header_rows:
        # Till this point all the code is custom for the website. Depending on the website source we have to write the 
        # code for traversing through the tags.
            if header_row.next_element.attrs.get('id') == 'production-data-table':
            # Read the body
                table_body = table.find('tbody')
                row_index = 0
                for body_row in table_body.find_all('tr'):
                    col_index = 0
                    body_cols = body_row.find_all('td')
                    for col in body_cols:
                        if col_index == 0:
                            table_data.loc[row_index, col_index] = col.get_text()
                        if col_index == 1:
                            url = col.find('a')
                            table_data.loc[row_index, col_index] = str(url)
                        if col_index == 2:
                            url = col.find('a')
                            table_data.loc[row_index, col_index] = str(url)
                        col_index += 1
                    row_index += 1
else:
    print('Error in reading page')

In [5]:
# Rename the column names of the newly formed dataframe
column_names = ['dataset_name', 'data_url_format', 'data_url_manual']
table_data.columns = column_names

In [6]:
# Extracting the URL from the column
table_data['data_url'] = table_data.data_url_format.str.extract('\"(.*)\"', expand = True)
table_data['data_desc_url'] = 'https://www.rrc.state.tx.us' + \
                                                        table_data.data_url_manual.str.extract('\"(.*)\"', expand = True)
table_data['data_format'] = table_data.data_url_format.str.extract('>(.*)<', expand = True)
table_data['data_desc_format'] = table_data.data_url_manual.str.extract('>(.*)<', expand = True)

# Dropping the column
table_data.drop(columns = ['data_url_format', 'data_url_manual'], inplace = True)

### ~Things to Do~
1. ~Clean dataset_name column~
    - ~Replace ( with spaces~
    - ~Replace ) with empty string~
    - ~Replace / with space~
    - ~Replace - with empty string~
    - ~Remove puntuations~
    - ~Remove all alphabet + number combination (6E, 7B etc)~
    - ~Remove numbers~
2. ~Remove all the rows that have NaN~
3. ~Create a new dataframe with the unique values of dataset_name column~
4. ~From data_url column~
    - ~Extract ftp://ftpe.rrc.texas.gov - from begining till the last /~
    - ~Extract ftpe.rrc.texas.gov - between // and /~
    - ~Extract folder name - after last /~
    - ~Create new columns for all the above~

In [7]:
# Replacing the below characters separately because of special consideration
table_data.dataset_name = table_data.dataset_name.str.replace('(', ' ').str.replace(')', '').str.replace('/', ' ')
# Replace charratcers like 8A, 6E etc
table_data.dataset_name = table_data.dataset_name.str.replace(r'[0-9][a-zA-Z]', '')
# Replace all numbers
table_data.dataset_name = table_data.dataset_name.str.replace(r'\W\d{1,}', '')
# Replace all punctuaitons
table_data.dataset_name = table_data.dataset_name.str.replace(r'[^\w\s]', '')
# Replace ' and '
table_data.dataset_name = table_data.dataset_name.str.replace(' and', '')
# Replace multiple consecutive whitespaces
table_data.dataset_name = table_data.dataset_name.str.replace(r'\s{1,}', ' ')
# Remove the last space
table_data.dataset_name = table_data.dataset_name.str.rstrip()
# Remove all the rows that have NaN in  any row
table_data = table_data.dropna()

In [8]:
# Dropping all the duplicate rows
table_data = table_data.drop_duplicates(keep='last').reset_index()

In [9]:
table_data['ftpe_gov_url'] = table_data.data_url.str.extract('//(.*)/', expand = True)
table_data['intial_ftp_url'] = table_data.data_url.str.extract('^(.*)/', expand = True)
table_data['folder_name'] = table_data.data_url.str.rsplit('/').str[-1]

In [10]:
table_data['file_url_list'] = ''

### Connecting to ftp page and getting the list of files from the webpage.

In [11]:
for index in table_data.index:
    with FTP(table_data.loc[index, 'ftpe_gov_url']) as ftp:
        ftp.login()
        ftp.cwd(table_data.loc[index, 'folder_name'])
        data = ftp.nlst()

#     path = table_data.loc[index, 'intial_ftp_url']
    # Using table_data.loc[index, 'file_url_list'] = [path + lnk for lnk in data] to insert the list in a column 
    # an gives and error 'ValueError: Must have equal len keys and value when setting with an iterable'. Therefore,
    # use .at. DataFrame.at accesses a single value for a row/column label pair.
    table_data.at[index, 'file_url_list'] = data

'230 Login successful.'

'250 Directory successfully changed.'

'230 Login successful.'

'250 Directory successfully changed.'

'230 Login successful.'

...

'250 Directory successfully changed.'

'230 Login successful.'

'250 Directory successfully changed.'

'230 Login successful.'

'250 Directory successfully changed.'

### ~Things to do~
1. ~Create the folder based on dataset_name~
2. ~Download the files from the file_url_list~

In [ ]:
base_folder = 'C:\\Users\\Admin\\Desktop\\Study\\NLP\\FTP_Files\\'
for index in table_data.index:
    download_folder = base_folder + table_data.loc[index, 'dataset_name']
    try:
        os.makedirs(download_folder)
    except FileExistsError:
        pass
    except OSError as e:
        raise
    
    url_list = table_data.loc[index, 'file_url_list']
    for lst_index in range(len(url_list)): 
        file = table_data.loc[index, 'data_url'] + '/' + url_list[lst_index]
        download_file_name = download_folder + '\\' + url_list[lst_index]

        with closing(req.urlopen(file)) as r:
            with open(download_file_name, 'wb') as f:
                shutil.copyfileobj(r, f)